In [ ]:
# Export from a gcloud shell.
# It does not include all columns in the table because a dump of all columns
# yielded a bad csv probably because of this known issue with exporting NULL values:
# https://cloud.google.com/sql/docs/mysql/known-issues#import-export

# gcloud sql export csv --escape="5C" software-usage-stats gs://logging-natcap/model_log_table-2022-06-28.csv --query="SELECT 'model_name', 'invest_release', 'invest_interface', 'system_full_platform_string', 'time' UNION SELECT model_name, invest_release, invest_interface, system_full_platform_string, time FROM model_log_table" --database=invest_model_usage

# From a local shell:
# gsutil cp gs://logging-natcap/model_log_table-2022-06-28.csv .

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas
import altair

from load_and_clean import load_and_clean_csv

pandas.set_option('display.max_rows', 100)

In [3]:
table_path = 'model_log_table-2022-06-28.csv'

In [4]:
df = load_and_clean_csv(table_path)

dropping 22888 rows for models we dont care about
remaining model counts:
sdr                                 54583
annual_water_yield                  41689
carbon                              38291
habitat_quality                     30209
ndr                                 27309
seasonal_water_yield                19450
recmodel_client                     17788
hra                                  9184
fisheries                            8556
pollination                          8230
urban_cooling_model                  7345
delineateit                          7331
coastal_vulnerability                6714
scenario_generator                   6452
coastal_blue_carbon                  5822
scenario_gen_proximity               3670
urban_flood_risk_mitigation          3349
scenic_quality                       2531
routedem                             1997
forest_carbon_edge_effect            1915
crop_production_percentile           1891
wind_energy                          1471
ro

# Workbench stats

In [6]:
release_date = pandas.to_datetime('2022-05-24', utc=True)
post_wb_runs = df[df['datetime'] >= release_date]
print(f'{post_wb_runs.shape[0]} invest runs since workbench release')
# wb_runs = post_wb_runs[post_wb_runs['invest_interface'].str.contains('Workbench')]
# print(f'including {wb_runs.shape[0]} Workbench 0.1.0-beta runs')
# wb_runs
v311 = post_wb_runs[post_wb_runs['invest_release'].str.contains('3.11.0')]
print(v311.shape[0])

4514 invest runs since workbench release
980


# Monthly counts by model

In [ ]:
# Aggregate to a time frequency so we can count runs per unit of time per model
frequency = 'M'
data = df.groupby([
    pandas.Grouper(key='datetime', freq=frequency),
    pandas.Grouper(key='model')]).size().reset_index(name='counts')
data.head()

In [ ]:
# For the benefit of plots, fill in 0s where no models were run
wide = data.pivot(index='datetime', columns='model', values='counts')
wide.fillna(0, inplace=True)
months_with_counts = len(wide)

# And in case there were months where no models were run
# And in case the first & last months are incomplete (assume they are)
# trim them off with offsets
begin = wide.index.min() + pandas.offsets.MonthBegin()
end = wide.index.max() - pandas.offsets.MonthEnd()
date_range = pandas.date_range(begin, end, freq=frequency)
wide = wide.reindex(date_range, fill_value=0)
print(f'complete data from {begin} to {end}')

In [ ]:
# Format data for altair
wide = wide.reset_index() # altair cannot plot indices, so move date to normal column
long = pandas.melt(wide, id_vars='index')
# long.head()

In [ ]:
def plot_model_counts_over_time(model_list, title=None):
    altair.data_transformers.disable_max_rows()
    selection = altair.selection_multi(fields=['model'], bind='legend')

    to_plot = long[long['model'].isin(model_list)]
    return (
        altair.Chart(to_plot).mark_line().encode(
            altair.X('index:T', axis=altair.Axis(format='%Y-%m'), title=None),
            altair.Y('value:Q', title='runs per month'),
            color=altair.Color('model', scale=altair.Scale(scheme='category10')),
            opacity=altair.condition(selection, altair.value(1), altair.value(0.2)),
            size=altair.value(1)
        ).properties(
            width=800,
            height=300,
            title=title
        ).add_selection(
            selection
        ).configure_axis(
            grid=False,
            labelFontSize=12,
            titleFontSize=14,
            titlePadding=15
        ).configure_legend(
            labelFontSize=12,
            title=None
        )
    )

In [ ]:
all_models_counts = long.groupby('index').sum().reset_index()
altair.Chart(all_models_counts).mark_line().encode(
    altair.X('index:T', axis=altair.Axis(format='%Y-%m'), title=None),
    altair.Y('value:Q', title='runs per month'),
    size=altair.value(1)
).properties(
    width=800,
    height=300,
    title='all models'
).configure_axis(
    grid=False
)

## These plots are interactive - click a series in the legend

In [ ]:
high_use_models = ['sdr',
                   'hydropower_water_yield',
                   'carbon',
                   'habitat_quality',
                   'ndr',
                   'seasonal_water_yield',
                   'recmodel_client',
                   'fisheries'
                  ]
plot_model_counts_over_time(high_use_models, 'high-use models')

In [ ]:
mid_use_models = ['pollination',
                  'hra',
                  'scenario_generator',
                  'coastal_vulnerability',
                  'urban_cooling_model',
                  'coastal_blue_carbon'
                 ]
plot_model_counts_over_time(mid_use_models, 'mid-use models')

In [ ]:
mid_low_use_models = [
    'delineateit',
    'scenario_gen_proximity',
    'urban_flood_risk_mitigation',
    'scenic_quality',
    'routedem',
    'crop_production_percentile',
    'wind_energy',
    'crop_production_regression',
]
plot_model_counts_over_time(mid_low_use_models, 'mid-low-use models')

In [ ]:
low_use_models = [
    'overlap_analysis',
    'wave_energy',
    'finfish_aquaculture',
    'fisheries_hst',
    'forest_carbon_edge_effect',
    'globio',
    'marine_water_quality_biophysical',
    'timber',
]
plot_model_counts_over_time(low_use_models, 'low-use models (including some that are already deprecated)')

In [ ]:
long[long['model'] == 'wave_energy']